## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mataura,NZ,2021-04-23 20:51:01,-46.1927,168.8643,45.00,95,53,1.99,broken clouds
1,1,Khatanga,RU,2021-04-23 20:51:01,71.9667,102.5000,33.06,97,100,21.21,overcast clouds
2,2,Tiksi,RU,2021-04-23 20:51:01,71.6872,128.8694,4.64,98,97,7.18,overcast clouds
3,3,Maputo,MZ,2021-04-23 20:51:02,-25.9653,32.5892,64.00,88,75,4.61,broken clouds
4,4,Kodiak,US,2021-04-23 20:51:02,57.7900,-152.4072,39.20,87,90,9.22,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
15,15,Avarua,CK,2021-04-23 20:51:05,-21.2078,-159.7750,82.40,74,75,16.11,broken clouds
20,20,Rikitea,PF,2021-04-23 20:51:06,-23.1203,-134.9692,79.09,67,100,17.78,overcast clouds
23,23,Tessalit,ML,2021-04-23 20:51:07,20.1986,1.0114,88.75,6,0,7.34,clear sky
28,28,Tahta,EG,2021-04-23 20:51:09,26.7693,31.5021,84.20,14,0,11.50,clear sky
29,29,Hilo,US,2021-04-23 20:51:09,19.7297,-155.0900,75.20,78,90,4.61,moderate rain
31,31,Mitu,CO,2021-04-23 20:48:20,1.1983,-70.1733,80.49,88,59,1.90,heavy intensity rain
33,33,Tual,ID,2021-04-23 20:51:10,-5.6667,132.7500,83.80,75,84,13.51,broken clouds
35,35,Bontang,ID,2021-04-23 20:51:11,0.1333,117.5000,75.16,94,72,7.72,broken clouds
42,42,Bambous Virieux,MU,2021-04-23 20:51:13,-20.3428,57.7575,78.80,78,75,10.36,light rain
45,45,Puerto Ayora,EC,2021-04-23 20:50:50,-0.7393,-90.3518,84.20,70,20,16.11,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                199
City                   199
Country                199
Date                   199
Lat                    199
Lng                    199
Max Temp               199
Humidity               199
Cloudiness             199
Wind Speed             199
Current Description    199
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
15,Avarua,CK,82.40,broken clouds,-21.2078,-159.7750,
20,Rikitea,PF,79.09,overcast clouds,-23.1203,-134.9692,
23,Tessalit,ML,88.75,clear sky,20.1986,1.0114,
28,Tahta,EG,84.20,clear sky,26.7693,31.5021,
29,Hilo,US,75.20,moderate rain,19.7297,-155.0900,
31,Mitu,CO,80.49,heavy intensity rain,1.1983,-70.1733,
33,Tual,ID,83.80,broken clouds,-5.6667,132.7500,
35,Bontang,ID,75.16,broken clouds,0.1333,117.5000,
42,Bambous Virieux,MU,78.80,light rain,-20.3428,57.7575,
45,Puerto Ayora,EC,84.20,few clouds,-0.7393,-90.3518,


In [8]:
# Set parameters to search for hotels with 5000 meters
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")     

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [15]:
# 7. Drop the rows where there is no Hotel Name.

clean_hotel_df = hotel_df[hotel_df["Hotel Name"] != ""]
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
15,Avarua,CK,82.40,broken clouds,-21.2078,-159.7750,Paradise Inn
20,Rikitea,PF,79.09,overcast clouds,-23.1203,-134.9692,Pension Maro'i
28,Tahta,EG,84.20,clear sky,26.7693,31.5021,El Salam Hotel
29,Hilo,US,75.20,moderate rain,19.7297,-155.0900,Hilo Hawaiian Hotel
33,Tual,ID,83.80,broken clouds,-5.6667,132.7500,Grand Vilia Hotel
...,...,...,...,...,...,...,...
658,Totness,SR,77.99,moderate rain,5.8833,-56.3167,STAATS LOGEERGEBOUW TOTNESS
660,Kieta,PG,78.91,overcast clouds,-6.2167,155.6333,Uruna Bay Retreat
661,Jacmel,HT,87.80,few clouds,18.2342,-72.5347,Hotel Florita
664,Texas City,US,75.20,haze,29.3838,-94.9027,Comfort Inn & Suites Texas City


In [16]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [17]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [18]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Call the figure to plot the data.
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))